In [3]:
import pandas as pd
import requests

from lxml import html
from bs4 import BeautifulSoup

In [16]:
def gongsa(inValue, inValue2):
    url="https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do"

    serviceKey1="4A0AC4AE-83B0-472F-8A15-6826490CBF79"
    # type	string	선택	검색결과 포맷(xml 또는 json)
    type1="xml"
    # pageNo	number	선택	페이지번호

    pageNo1= str(inValue)
    # numOfRows	number	선택	한페이지 결과수 / 함수의 첫 번째 파라미터 값

    numOfRows1= str(inValue2)
    # sortField	string	선택	정렬하고자 하는 출력변수명  / 함수의 두 번째 파라미터 값
    
    sortField1=""
    # sortOrder	string	선택	정렬순서(ASC 또는 DESC)
    
    sortOrder1=""
    # searchCcwYn	string	선택	준공여부(0:진행,1:준공)
    
    searchCcwYn1=""
    # searchOrcd	string	선택	발주기관(코드집 참고 : 기관코드)
    
    searchOrcd1=""
    # searchCwkNm	string	선택	공사명
    
    searchCwkNm1=""
    targetUrl=url+"?&serviceKey="+serviceKey1+"&type="+type1+"&pageNo="+pageNo1+\
    "&numOfRows="+numOfRows1+"&sortField="+sortField1+"&sortOrder="+sortOrder1+"&searchCcwYn="+\
    searchCcwYn1+"&searchOrcd="+searchOrcd1+"&searchCwkNm="+searchCwkNm1



    # targetUrl = "https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do?serviceKey=49D453F4-2B1E-4B9D-BC2A-4CC9EE27EE4D&type=xml&pageNo=1&numOfRows=10&searchCwkNm=7"

    resp = requests.get(targetUrl)

    resp.encoding = "utf-8"

    resp.text
    
    # BeautifulSoup으로 떠온 결과를 변수 xmlData에 담는다.
    xmlData = BeautifulSoup(resp.text,"lxml-xml")
    
    #item태그를 모두 찾는다.
    #각 item 태그는 하나의 리스트로 한 행씩 잘라져있다. (한개의 item태그에는 하나의 공사 정보가 모두 들어있다.)

    xmlData.findAll("item")
    
    #한 리스트씩 담아야 하므로 xmlRows로 변수 지정. 행 단위로 수집 
    #전체 리스트의 개수는 xmlRowsLen으로 변수 지정.
    
    xmlRows = xmlData.findAll("item")
    xmlRowsLen = len(xmlRows)

    #각 행의 <컬럼 값>을 담을 빈 리스트 지정
    eachColumnList = []
    
    #각 행의 컬럼 값을 모아서 하나의 행을 만들고, 그 행들을 순차적으로 넣을 최종 리스트 생성. 초기 값은 빈 값.
    RowList = []

    # 각 행의 컬럼 값들을 모아 eachColumnList에 담은 뒤, 순차적으로 RowList로 담는다. 
    # RowList에 append 시킨 뒤, 새로운 값을 받아 append 하기 위해 eachColumnList를 비워주는 로직을 넣는다.
    # 컬럼의 개수에 맞춰서 담기 위해 두 번째 for문에서는 columns의 길이만큼 반복한다.
    
    for j in range(0,xmlRowsLen):
        columns = xmlRows[j].findAll()
        for i in range(0,len(columns)):
            eachColumnValue = columns[i].text
            eachColumnList.append(eachColumnValue)
        RowList.append(eachColumnList)
        eachColumnList = []
    
    # 컬럼명 지정
    columnHead = ["순번","발주청사업자등록번호","현장번호","공사번호","공사명","기관코드","기관코드명","노선(하천)코드","노선(하천)명","행정구역명","사업종류코드","사업종류명","사업종류차수","착공일자","준공일자","준공예정일자","준공여부(0:진행,1:준공)","준공여부명(0:진행,1:준공)","사업분야코드","사업분야명","공사구간"]

    # RowList와 columnHead를 DataFrame을 활용해서 묶는다.
    finalDf = pd.DataFrame(RowList,columns = columnHead)

    # csv파일로 저장하기위해 경로를 설정하고, 각 페이지를 표시한다. 함수의 첫 번째 파라미터 값을 파일번호로 사용한다.
    finalDf.to_csv('../dataset/GongSa'+str(inValue)+'.csv',index=False)

for i in range(1,14):
    gongsa(i,100)